In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re    
import glob
import numpy as np         
import tensorflow as tf    
import os
from sklearn.model_selection import train_test_split

txt_file_path = ("/content/drive/MyDrive/Colab Notebooks/lyricist/*")

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
  with open(txt_file, "r") as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187089
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [3]:
for idx, sentence in enumerate(raw_corpus):

    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue 
    if idx >9: break 
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'      
    
    return sentence


In [5]:
corpus = []

for sentence in raw_corpus:
  if len(sentence) == 0: continue
  if sentence[-1] == ":": continue

  corpus.append(preprocess_sentence(sentence))

corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[    2   304    28 ...     0     0     0]
 [    2   221    13 ...     0     0     0]
 [    2    24    17 ...     0     0     0]
 ...
 [    2    13   440 ...     0     0     0]
 [    2    26    17 ...     0     0     0]
 [    4  1644 10643 ...     4  4669     3]] <keras_preprocessing.text.Tokenizer object at 0x7f7408576d50>


In [7]:
for idx in tokenizer.index_word:
  print(idx, ":", tokenizer.index_word[idx])

  if idx>=10: break


1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2  304   28   99 4812    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[ 304   28   99 4812    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=2)


In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset


<BatchDataset element_spec=(TensorSpec(shape=(256, 19), dtype=tf.int32, name=None), TensorSpec(shape=(256, 19), dtype=tf.int32, name=None))>

In [11]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 19), dtype=tf.int32, name=None), TensorSpec(shape=(256, 19), dtype=tf.int32, name=None))>

In [12]:
print(steps_per_epoch)

549


In [13]:
print("Source Train", enc_train.shape)
print("Target Train", dec_train.shape)

Source Train (140600, 19)
Target Train (140600, 19)


# 인공지능 만들기

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
        
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)


In [15]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 19, 15001), dtype=float32, numpy=
array([[[-4.65645018e-04,  2.70456192e-04,  2.13819541e-04, ...,
         -1.05143808e-04,  4.81698735e-05, -1.27589170e-04],
        [-7.87454192e-04,  3.09480733e-04,  2.78743508e-04, ...,
          7.78614121e-05,  4.11331894e-05, -1.31556852e-04],
        [-9.02497210e-04,  1.92304447e-04,  3.82778584e-04, ...,
          2.07989706e-05, -2.34694395e-04,  1.05635963e-05],
        ...,
        [-2.26701517e-03, -3.58482986e-03, -1.29570928e-03, ...,
         -8.50679353e-04, -4.80103481e-04,  2.04198644e-03],
        [-2.81005981e-03, -4.03085677e-03, -1.37614144e-03, ...,
         -7.74373475e-04, -4.28662956e-04,  1.99246290e-03],
        [-3.32592777e-03, -4.47462406e-03, -1.45621598e-03, ...,
         -6.66609907e-04, -4.10856941e-04,  1.90932804e-03]],

       [[-4.65645018e-04,  2.70456192e-04,  2.13819541e-04, ...,
         -1.05143808e-04,  4.81698735e-05, -1.27589170e-04],
        [-6.06664282e-04,  7.26586150e-04,  5

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  7680512   
                                                                 
 lstm (LSTM)                 multiple                  20979712  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  30737049  
                                                                 
Total params: 92,959,897
Trainable params: 92,959,897
Non-trainable params: 0
_________________________________________________________________


In [17]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer, )
model.fit(dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 208s 372ms/step - loss: 2.7571 - val_loss: 2.4540
Epoch 2/10
549/549 [==============================] - 204s 372ms/step - loss: 2.2802 - val_loss: 2.2024
Epoch 3/10
549/549 [==============================] - 204s 372ms/step - loss: 2.0212 - val_loss: 2.0546
Epoch 4/10
549/549 [==============================] - 203s 370ms/step - loss: 1.7749 - val_loss: 1.9469
Epoch 5/10
549/549 [==============================] - 203s 370ms/step - loss: 1.5469 - val_loss: 1.8756
Epoch 6/10
549/549 [==============================] - 203s 369ms/step - loss: 1.3437 - val_loss: 1.8265
Epoch 7/10
549/549 [==============================] - 203s 370ms/step - loss: 1.1692 - val_loss: 1.7979
Epoch 8/10
549/549 [==============================] - 203s 370ms/step - loss: 1.0244 - val_loss: 1.7897
Epoch 9/10
549/549 [==============================] - 203s 370ms/step - loss: 0.9117 - val_loss: 1.8007
Epoch 10/10
549/549 [==============================] - 203s 370m

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  

         
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

     
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   

In [19]:
generate_text(model, tokenizer, init_sentence="<start> money")

'<start> money and blood dont mix like dicks and no bitch <end> '

#회고
이번에 학습을 시키면서 시간이 오래걸리는 것을 보면서 와.. 이제 드디어 시작이구나 진짜 학습을 시키는거구나를 느꼈다.  
처음에 학습을 시켰을때는 embedding_size를 1024로 hidden_size를 2048로 하여 학습을 진행하였으나 2.3 이하로 내려가지 않아서 당황했다. 그래서 embedding_size와 hidden_size를 엄청나게 늘렸으나 오히려 값이 올라서 이게 뭔가 싶었다. 그래서 다른분들께 조언을 구하여 단어장의 갯수를 12000에서 15000으로 늘렸고, 2000까지 늘렸던 embedding_size를 512로 낮추고 hidden_size를 2048로 다시 줄렸다. 그 결과 1.8까지 내려갈 수 있었다.  

이번 익스를 진행하면서도 코드를 짜는 것이 여전히 어려웠다. 개념적으로 부족하다고 판단하여서 이 부분에 대한 개념을 반복 숙달 해야할 것 같다.